In [1]:
# Load libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from time import time
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import warnings
import math
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# load default data frames
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
profiles_df = pd.read_csv("profiles.csv")
artists_df = pd.read_csv("artists.csv")
user_median_df = pd.read_csv("user_median.csv")
global_median_df = pd.read_csv("global_median.csv")

In [3]:
# load genre data frames and do some cleaning
artists_with_1genre_df = pd.read_csv("artists_with_genres.csv")
artists_with_5genres_df = pd.read_csv("artists_with_top_5_genres.csv")
artists_with_5genres_df.rename(columns = {"num_votes5":"extra"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre5":"num_votes5"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes4":"genre5"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre4":"num_votes4"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes3":"genre4"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre3":"num_votes3"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes2":"genre3"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre2":"num_votes2"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes1":"genre2"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre1":"num_votes1"}, inplace = True)
artists_with_5genres_df.rename(columns = {"name":"genre1"}, inplace = True)
artists_with_5genres_df.rename(columns = {"artist":"name"}, inplace = True)

In [4]:
# Train - Test Split on profiles to ensure users are in one group or another
train, test = train_test_split(train_df, test_size = .1)
train, valid = train_test_split(train, test_size = .1)

In [5]:
# add primary genre to train, valid and test
train["genre1"] = np.nan
train.set_index("artist", inplace=True)
train.update(artists_with_5genres_df)
train.reset_index(inplace=True)

valid["genre1"] = np.nan
valid.set_index("artist", inplace=True)
valid.update(artists_with_5genres_df)
valid.reset_index(inplace=True)

test["genre1"] = np.nan
test.set_index("artist", inplace=True)
test.update(artists_with_5genres_df)
test.reset_index(inplace=True)

In [6]:
def calc_user_avgs(df):
    grp = df.groupby('user')
    user_median = grp.plays.median()
    user_mean = grp.plays.mean()
    df2 = df.copy()
    df2.set_index('user', inplace=True)
    df2['user median'] = user_median
    df2['user mean'] = user_mean
    df2.reset_index(inplace=True)
    df2.drop("artist", axis=1,inplace=True)
    df2.drop("plays",axis=1, inplace = True)
    df2.drop("genre1",axis=1, inplace = True)
    return df2

In [7]:
def calc_artist_avgs(df):
    grp = df.groupby('artist')
    artist_median = grp.plays.median()
    artist_mean = grp.plays.mean()
    df2 = df.copy()
    df2.set_index('artist', inplace=True)
    df2['artist median'] = artist_median
    df2['artist mean'] = artist_mean
    df2.reset_index(inplace=True)
    df2.drop("user", axis=1,inplace=True)
    df2.drop("plays",axis=1, inplace = True)
    df2.drop("genre1",axis=1, inplace = True)
    return df2

In [8]:
def calc_genre_avgs(df):
    grp = df.groupby('genre1')
    genre_median = grp.plays.median()
    genre_mean = grp.plays.mean()
    df2 = df.copy()
    df2.set_index('genre1', inplace=True)
    df2['genre median'] = genre_median
    df2['genre mean'] = genre_mean
    df2.reset_index(inplace=True)
    df2.drop("user", axis=1,inplace=True)
    df2.drop("plays",axis=1, inplace = True)
    df2.drop("artist",axis=1, inplace = True)
    return df2

In [9]:
# compute various averages for use later
user_avgs   = calc_user_avgs(train)
user_avgs.set_index("user",inplace=True)
user_avgs.drop_duplicates(inplace=True)

artist_avgs = calc_artist_avgs(train)
artist_avgs.set_index("artist",inplace=True)
artist_avgs.drop_duplicates(inplace=True)

genre_avgs  = calc_genre_avgs(train)
genre_avgs.set_index("genre1",inplace=True)
genre_avgs.drop_duplicates(inplace=True)

user_genre_median = train.groupby(["user","genre1"]).median().reset_index()
user_genre_median.rename(columns={'plays':'user genre median'}, inplace=True)
user_genre_median.set_index(['user','genre1'],inplace=True)

global_median = global_median_df.iloc[0,1]

In [10]:
#pivot training data to have one row per user
train = train.pivot(index='user',columns='artist',values='plays').reset_index()
train = train.fillna(0)

In [11]:
# use this function in APPLY for a df
def knn_artist_predict(test_row,train,nk,weight):
    user = test_row[1]
    artist = test_row[0]

    tmp = train[train[test_row.artist] > 0].set_index("user")
    y = tmp[artist]
    X = tmp.drop(artist,axis=1)
    
    neigh = KNeighborsRegressor(n_neighbors=nk, weights=weight)
    neigh.fit(X,y)
    
    to_test = train[train['user'] == user].set_index('user').drop(artist,axis=1)
    prediction = neigh.predict(to_test)[0]
    
    return prediction

In [12]:
# use this function in APPLY for a df
def knn_profile_predict(test_row,train,nk,weight):
    #######
    # insert knn using profile info and that genre and/or artist
    #######
    return AE

In [52]:
# initialize columns
quick = valid.iloc[:10,:]
quick['knn'] = np.nan
quick['user genre median'] = np.nan
quick['user median'] = np.nan
quick['genre median'] = np.nan
quick['global median'] = np.nan
quick.head(1)

,artist,user,plays,genre1,knn,user genre median,user median,genre median,global median
0,9a3bf45c-347d-4630-894d-7cf3e8e0b632,50890bcb3e094802c7d0603057de998425419661,46,latin rock,NaN,NaN,NaN,NaN,NaN


In [53]:
# add knn artist 
quick.knn = quick.apply(lambda x: knn_artist_predict(x,train,3,'uniform'),axis=1)
quick.head(1)

,artist,user,plays,genre1,knn,user genre median,user median,genre median,global median
0,9a3bf45c-347d-4630-894d-7cf3e8e0b632,50890bcb3e094802c7d0603057de998425419661,46,latin rock,15.666667,NaN,NaN,NaN,NaN


In [54]:
# add global median
quick['global median'] = global_median
quick.head(1)

,artist,user,plays,genre1,knn,user genre median,user median,genre median,global median
0,9a3bf45c-347d-4630-894d-7cf3e8e0b632,50890bcb3e094802c7d0603057de998425419661,46,latin rock,15.666667,NaN,NaN,NaN,118


In [55]:
# add user median
quick.set_index('user',inplace=True)
quick.update(user_avgs)
quick.reset_index(inplace=True)
quick.head(1)

,user,artist,plays,genre1,knn,user genre median,user median,genre median,global median
0,50890bcb3e094802c7d0603057de998425419661,9a3bf45c-347d-4630-894d-7cf3e8e0b632,46,latin rock,15.666667,NaN,21,NaN,118


In [56]:
# add genre median
quick.set_index('genre1',inplace=True)
quick.update(genre_avgs)
quick.reset_index(inplace=True)
quick.head(1)

,genre1,user,artist,plays,knn,user genre median,user median,genre median,global median
0,latin rock,50890bcb3e094802c7d0603057de998425419661,9a3bf45c-347d-4630-894d-7cf3e8e0b632,46,15.666667,NaN,21,71,118


In [58]:
# add genre median
quick.set_index(['user','genre1'],inplace=True)
quick.update(user_genre_median)
quick.reset_index(inplace=True)
quick.head(10)

,user,genre1,artist,plays,knn,user genre median,user median,genre median,global median
0,50890bcb3e094802c7d0603057de998425419661,latin rock,9a3bf45c-347d-4630-894d-7cf3e8e0b632,46,15.666667,NaN,21.0,71,118
1,a51b1b5aca3bc0ab8edc42529ae8a72bee793d78,alternative metal,b15ebd71-a252-417d-9e1c-3e6863da68f8,383,196.333333,281,275.5,128,118
2,7deda26d9d9188ba1417cf5cc92382e851c513f2,soul,d8df96ae-8fcf-4997-b3e6-e5d1aaf0f69e,15,4.666667,NaN,23.0,91,118
3,1350ddd9a60af9dc7d24c5a73cc0ae9b6fd4cb53,alternative rock,d8354b38-e942-4c89-ba93-29323432abc3,32,16.333333,38,65.5,121,118
4,1033355727c2722e279bfeade9142213b532a223,emo-pop,516cef4d-0718-4007-9939-f9b38af3f784,65,196.333333,77,138.5,126,118
5,5fd72224a2f610553a50c85ec54023561fc2a264,art rock,aa7a2827-f74b-473c-bd79-03d065835cf7,14,4.333333,NaN,NaN,112,118
6,21a3b4ef70d612d7b4d0d048c71d54e205e02b3a,british,77f049ad-f469-4ad1-8283-7a2606a6722e,130,41.666667,50,79.5,99,118
7,b5194e9b284b41b5d51a2a8c137b73b04273f958,progressive death metal,c14b4180-dc87-481e-b17a-64e4150f90f6,95,61.000000,NaN,NaN,188,118
8,e50bbd1f9e34dade615681750b3c21f9207dc0b1,alternative rock,847e8284-8582-4b0e-9c26-b042a4f49e57,45,30.333333,NaN,46.5,121,118
9,f050d21ec4b1d2b099e7da3647e03fbd6ced9820,mod,17b53d9f-5c63-4a09-a593-dde4608e0db9,138,19.666667,NaN,92.5,114,118


In [60]:
# calculate mean of our predictions
quick = quick.fillna(0)
quick['mean'] = (quick['knn'] + quick['user genre median'] + quick['user median'] + quick['genre median'] + quick['global median'])/5

In [61]:
# calculate Abs Error of predictions
quick['AE'] = np.abs(quick['plays']-quick['mean'])
quick.head()

,user,genre1,artist,plays,knn,user genre median,user median,genre median,global median,mean,AE
0,50890bcb3e094802c7d0603057de998425419661,latin rock,9a3bf45c-347d-4630-894d-7cf3e8e0b632,46,15.666667,0,21.0,71,118,45.133333,0.866667
1,a51b1b5aca3bc0ab8edc42529ae8a72bee793d78,alternative metal,b15ebd71-a252-417d-9e1c-3e6863da68f8,383,196.333333,281,275.5,128,118,199.766667,183.233333
2,7deda26d9d9188ba1417cf5cc92382e851c513f2,soul,d8df96ae-8fcf-4997-b3e6-e5d1aaf0f69e,15,4.666667,0,23.0,91,118,47.333333,32.333333
3,1350ddd9a60af9dc7d24c5a73cc0ae9b6fd4cb53,alternative rock,d8354b38-e942-4c89-ba93-29323432abc3,32,16.333333,38,65.5,121,118,71.766667,39.766667
4,1033355727c2722e279bfeade9142213b532a223,emo-pop,516cef4d-0718-4007-9939-f9b38af3f784,65,196.333333,77,138.5,126,118,131.166667,66.166667


In [62]:
# calculate MAE
quick.AE.mean()

51.65333333333334

In [ ]:
# weighted ensemble?
from sklearn import linear_model
stacking= linear_model.LinearRegression()

X = np.vstack([quick.knn.values,quick['user genre median'].values])
X = np.vstack([X,quick['user median']])
X = np.vstack([X,quick['genre median']])
X = np.vstack([X,quick['global median']])
X = X.T
X.shape

y = quick['plays'].values

In [ ]:
stacking.fit(X,y)

In [ ]:
y_pred = stacking.predict(X)

In [ ]:
np.mean(np.abs(y-y_pred))